In [12]:
!nvidia-smi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Mon Oct 28 20:00:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4090        Off | 00000000:19:00.0 Off |                  Off |
|  0%   35C    P8              20W / 450W |   2471MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [13]:
import tensorflow.compat.v1 as tf1
import os

# GPU 번호 지정 '0'은 1번 GPU,'1'은 2번 GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
config = tf1.ConfigProto()

# GPU 사용량 지정 0.03은 3%를 사용하겠다는 뜻
# 가능한 3% 이하로 사용할 것
config.gpu_options.per_process_gpu_memory_fraction = 0.5
session = tf1.Session(config=config)

I0000 00:00:1730113223.474288   45171 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 12108 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:19:00.0, compute capability: 8.9
I0000 00:00:1730113223.474967   45171 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 12108 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:1a:00.0, compute capability: 8.9
I0000 00:00:1730113223.475573   45171 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 12108 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:67:00.0, compute capability: 8.9
I0000 00:00:1730113223.476080   45171 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 12107 MB memory:  -> device: 3, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:68:00.0, compute capability: 8.9


In [5]:
# 여러개의 PDF 불러오기
import os
from langchain.document_loaders import PyPDFLoader
# from langchain.document_loaders import PDFPlumberLoader
from concurrent.futures import ThreadPoolExecutor
root_directory = "D:/nauri/nauri/nbot/document"
batch_size = 100
pdfdocs = []

In [6]:
# PDF 파일을 처리하는 함수
def process_pdf_batch(pdf_files):
    batch_docs = []
    for pdf_file_path in pdf_files:
        # pdf_loader = PDFPlumberLoader(pdf_file_path)
        pdf_loader = PyPDFLoader(pdf_file_path)
        batch_docs.extend(pdf_loader.load())
    return batch_docs

In [7]:
# 처리할 PDF 파일 리스트 생성
pdf_files_to_process = []
for root, dirs, files in os.walk(root_directory):
    pdf_files_to_process.extend([os.path.join(root, file) for file in files if file.lower().endswith(".pdf")])

In [9]:
# ThreadPoolExecutor를 사용한 병렬 처리
with ThreadPoolExecutor() as executor:
    total_files = len(pdf_files_to_process)
    processed_files = 0

    # Iterate through the PDF files in batches
    for i in range(0, total_files, batch_size):
        batch = pdf_files_to_process[i:i+batch_size]
        batch_docs = list(executor.map(process_pdf_batch, [batch]))
        for batch_result in batch_docs:
            pdfdocs.extend(batch_result)
            processed_files += len(batch)
            print(f"Processed {processed_files} / {total_files} files")

Processed 1 / 1 files


======================= 불러온 PDF를 임베딩 모델을 통해 Vector DB 제작 =======================

In [10]:
from langchain_community.embeddings import HuggingFaceEmbeddings

# cache_folder를 사용해 원하는 위치에 모델을 저장할 수 있음.
cache_folder = 'D:/nauri/nauri/nbot/model'



# 문장을 임베딩으로 변환하고 벡터 저장소에 저장
embeddings = HuggingFaceEmbeddings(
    model_name='intfloat/multilingual-e5-base',
    cache_folder=cache_folder,
    encode_kwargs={'normalize_embeddings':True}
)

In [12]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

def create_vector_store(split_type: str, threshold: int, docs, embeddings) -> FAISS:
    text_splitter = SemanticChunker(
        embeddings=embeddings,
        breakpoint_threshold_type=split_type,
        breakpoint_threshold_amount=threshold,
        min_chunk_size=150,
        buffer_size=3
    )
    split_docs = text_splitter.split_documents(docs)


    vector_store = FAISS.from_documents(
        split_docs,
        embedding=embeddings,
        distance_strategy=DistanceStrategy.COSINE
    )

    return vector_store


def generate_db_path(filename: str) -> str:
    base_path = 'D:/nauri/nauri/nbot/db'
    return f"{base_path}/{filename}"

In [13]:
pdfdocs

[Document(metadata={'producer': 'Hancom PDF 1.3.0.534', 'creator': 'Hwp 2018 11.0.0.1623', 'creationdate': '2025-03-28T15:46:41+09:00', 'author': 'Administrator', 'moddate': '2025-03-28T15:46:41+09:00', 'pdfversion': '1.4', 'source': 'D:/nauri/nauri/nbot/document\\참고-제안요청서(차세대 e-HRD 시스템 구축)_수정4.pdf', 'total_pages': 110, 'page': 0, 'page_label': '1'}, page_content='제 안 요 청 서과 제 명차세대 e-HRD 시스템 구축주관기관농촌진흥청 농촌인적자원개발센터\n2024. 5.\n구분부서직위성명연락처TELFAX사업책임자교육훈련기획과과장신명경063-238-1820063-238-1825사업담당자교육훈련기획과농촌지도사곽지훈063-238-1850농촌지도사김국현063-238-1851'),
 Document(metadata={'producer': 'Hancom PDF 1.3.0.534', 'creator': 'Hwp 2018 11.0.0.1623', 'creationdate': '2025-03-28T15:46:41+09:00', 'author': 'Administrator', 'moddate': '2025-03-28T15:46:41+09:00', 'pdfversion': '1.4', 'source': 'D:/nauri/nauri/nbot/document\\참고-제안요청서(차세대 e-HRD 시스템 구축)_수정4.pdf', 'total_pages': 110, 'page': 1, 'page_label': '2'}, page_content='•••• 목  차••••Ⅰ. 사업개요    1. 사업일반 ··························································

In [15]:
vector_store = create_vector_store(
    split_type="percentile",
    threshold=30,
    docs=pdfdocs,
    embeddings=embeddings
)

vector_store.save_local(generate_db_path("percentile"))



In [16]:
db_load = FAISS.load_local(generate_db_path("percentile"), embeddings, allow_dangerous_deserialization=True)
db_load

======================= 생성한 DB를 사용한 BM25용 pkl 파일 제작 =======================

In [ ]:
db3 = FAISS.load_local('/home/hjkll25/LM_Project/DB/SemDB40', embeddings, allow_dangerous_deserialization=True)

In [ ]:
from langchain_core.documents import Document
from konlpy.tag import Okt
import os
import pickle

# KoNLPy Okt 형태소 분석기 초기화
okt = Okt()

# 텍스트 전처리 함수 정의
def preprocess_text(text):
    # 형태소 분석을 통해 명사와 동사만 추출
    tokens = okt.nouns(text) + [word for word, pos in okt.pos(text) if pos.startswith('V')]
    # 토큰을 공백으로 연결하여 반환
    return ' '.join(tokens)

# 저장된 전처리된 문서 파일 경로
preprocessed_docs_path = 'preprocessed_docs.pkl'

if os.path.exists(preprocessed_docs_path):
    # 파일이 존재하면 불러오기
    with open(preprocessed_docs_path, 'rb') as file:
        preprocessed_docs = pickle.load(file)
    print(f"Loaded {len(preprocessed_docs)} preprocessed documents from file.")
else:
    # 파일이 없으면 전처리 수행
    print("No preprocessed file found. Starting preprocessing...")
    faiss_docs = [
        Document(page_content=doc.page_content, metadata=doc.metadata)
        for doc in vectorstore.docstore._dict.values()
    ]
    preprocessed_docs = [
        Document(page_content=preprocess_text(doc.page_content), metadata=doc.metadata)
        for doc in faiss_docs
    ]
    # 전처리된 문서들을 파일에 저장
    with open(preprocessed_docs_path, 'wb') as file:
        pickle.dump(preprocessed_docs, file)
    print(f"Saved {len(preprocessed_docs)} preprocessed documents to file.")

In [ ]:
from langchain_core.documents import Document
import os
import pickle

preprocessed_docs_path = 'preprocessed_docs.pkl'

if os.path.exists(preprocessed_docs_path):
    # 파일이 존재하면 불러오기
    with open(preprocessed_docs_path, 'rb') as file:
        preprocessed_docs = pickle.load(file)
else:
    # 파일이 없으면 전처리 수행
    faiss_docs = [
        Document(page_content=doc.page_content, metadata=doc.metadata)
        for doc in vectorstore.docstore._dict.values()
    ]
    # 전처리된 문서들을 파일에 저장
    with open(preprocessed_docs_path, 'wb') as file:
        pickle.dump(preprocessed_docs, file)